# Social Data Mining 2016 - Practical 4: Know your Performance

Any machine learning task can be formally evaluated by using the labels obtained from the feature that was selected as target for prediction, and comparing them to the predicted labels by an algorithm. Previously we did this by simply looking at the error (deviation from the true values), or accuracy scores (percentage of correctly predicted instances). 

As our point is generalization, training and evaluating (or testing) on the same data will not give us a good sense about how well the model will perform on totally new data (which is what we are interested in). Rather, the results would merely hold for reproduction; showing how well a model with perfect information would be able to reproduce exactly the same labels. The common result of this is that intuitively uninformative feautres (unique for each instance, does not contain any information about the feature we are trying to predict) might yield incredibly high results, while intuitively informative features might not. As this is counter-intuitive, it is correct to deem this behaviour as wrong. Uninformative features should NOT yield high scores (unless they have some surprising characteristic, which is unlikely), and high evaluation scores, or very low erros, in general should not be trusted at face-value.

### Testing Models

In a realistic setting, we would want to leave some of our data 'unseen' by the algorithm. We provide it with only the feature vectors (no labels), and hope that it will guess the label correctly. Now, in that setting, if we obtain a high score then we might conclude that our algorithm is performing well, and we can actually interpret the actual contribution of features (for some models) to this score. Still, even the interpretation of the performance on the test set should be considered in the light of (i) the size and quality of your data, and (ii) the hyper-parameter settings of your model.

One of the more common ways of making sure that the model is provided with some unseen data is by simply chopping off a part of your data, and leaving it aside for testing. As such, you have A) a big training set, B) a small test test. The actual proportions of these sets depend on the amount of data that you have, but in general it's good to leave at least 20% for testing. These proportions are **not set in stone**, and therefore you cannot just say "always leave 20% for testing". Treat it as a starting point.

### Baselines

In this setting we have some guarantee that the model is tested fairly. However, we are still unsure if it has actually learned something. The notion of learning is a bit simplified for our purposes: in general, we want to either i) outperform state-of-the-art models doing the same task (otherwise no need for us to train our own), or, if there is not such a model, ii) beat some simple model, or a very stupid approach to prediction. The latter is known as a _baseline_. We have seen this before when using the `Mean Learner`: a model that only looks at the average target value while training, and will predict this for all new data that it sees. For classification, we use a similar variant: the `Majority` baseline. At training time, it maps the distribution over labels (so the label and their associated frequencies), and at test time, it always predicts the most common label.

Both of these baselines perform very well under certain circumstances: if the target feature is normally distributed in case of regression, and if there's one very common class in the case of classification.

### Different Sets for Different Purposes

As we've seen up until now, any Data Mining routine starts with data; as we're using pre-made datasets the first step is determining the goal of the dataset, informing yourself about the features, preprocessing, spotting any contamination in your featureset and setting up a prediction task for yourself. To know how well you are doing while running your models we can discern *up to* four types of splits in a given dataset:

- Training set
- Validation (or development) set
- Testing set

The size of these with respect to the whole dataset usually differs, and if you are using data for an existing prediction task, usually you are already provided with a test set. In general, popular splits are (train / validation / test) 80/10/10 or 50/25/25 (given enough data).

### Validation

The procedure of testing performance would then be as follows: we train our model on the training set, and evaluate using the validation set. We tune the hyper-parameters of our classifiers (like $k$ for $k$-NN) to maximize the score on our validation set (and generally between our training and validation set).

The intuition behind this is as follows: since we are trying to predict new data (not in our set as this moment), we want to simulate this in some way with our completely unseen test set. If we would tune the hyper-parameters based on the result on the test set, the optimal hyper-parameter set for this piece of data is now know to us. However, if we then get actual new data, it might turn out that this _wasn't_ the best hyper-parameter setting, and we now perform very poorly. So, effectively, we would **not** know how the model performs on new data. If we use a validation set to determine the effect of tuning the hyper-parameters, we can determine an optimal setting, and then after have a fair, unseen evaluation round on the test data, so that we have a correct measure of how the model would perform on new data

### Evaluation Metrics

So how do we meaningfully interpret the performance of our model? 

So far, we have looked at $R^2$ and RMSE for regression. For classifcation, we have briefly mentioned accuracy; the percentage of correctly classified instances. Let's go into a bit more detail on it here.

#### Classification Metrics

For this example, let's assume that we have a dataset with a lot of dogs (5000), and some cats (200). We want to see if we can distinguish a `cat` from a `dog`. As such, we have two classes: `cat`, and `dog`. Cat is the minority class here, so we are actually interested to see how well we will do on classifying cats amongst a lot of dogs (because a majority baseline would get a pretty high score for dogs). As such, `cat` will be our **Positive** instance (that which we are interested in doing well at), and `dog` **Negative**.

Consider the example below:


| true label ($y$) | predicted label ($\hat{y}$) |
| ---------------- | --------------------------- |
| dog 	           | cat                         |
| cat 	           | dog                         |
| dog 	           | dog                         |
| cat 	           | cat                         |
| dog 	           | dog                         |
| dog 	           | cat                         |
| dog 	           | cat                         |
| cat 	           | cat                         |
| cat 	           | cat                         |
| cat 	           | cat                         |

#### Confusion Matrix

This is generally the first thing we look at. A matrix representation of the postive instances we predicted correctly (true positives), the positive instances we predict incorrectly (false negatives), the negative instances we predict incorrectly (false positives), and the negative instances we predicted correctly (true negatives). The notion here is that you treat it from the class point of view: if we predict some instance to be a negative class incorrectly (false), it's a false negative. So false negatives should actually have been positive. False positives should have been negative.

We represent it like this:

                     predicted
                  | Cat  | Dog |
           | ---- | ---- | --- |
    true   | Cat  |  tp  | fn  |
    true   | Dog  |  fp  | tn  |
    
    
Where tp is true positive, fn is false negative, etc.

#### Accuracy

The amount of correctly predicted examples:

$\text{Accuracy} = \frac{TP + TN}{TP + TN + FN + FP}$

#### Recall

Will tell you something about how many of the instances that you **wanted to classify** you actually classified correctly. So in our example, if we managed to classify all cats as actual cats (even if we classified some dogs as a cat) our recall is 100%. This distinguishes itself from accuracy, because accuracy says something about how well we classify ALL our classes. Say that we classified 100 cats as cat, but 100 **cats as dog**, our recall is 50%.

$\text{Recall} = \frac{\text{TP}}{\text{TP + FN}}$ 

#### Precision

Will tell you something about how many instances you **have classified** as being Positive (i.e. cat here), were actually a cat. In our example, if managed to not classify any `dog`s as `cat`s (even if we only classfied two instances as being a cat), our precision is 100%. Say that we classified 100 cats as cat, but also 100 **dogs as cat**, our precision is 50%. So:

$\text{Precision} = \frac{\text{TP}}{\text{TP + FP}}$ 


**NOTE:** We currently only explain this in a binary prediction task (with only 2 labels, so either yes or no, dog or cat). In reality you probably have more labels, just make sure that you understand it in the binary scenario.

#### $F_1$ Measure

These two metrics therefore trade off two properties of our prediction process: how many instances that we want to detect as Positive do we actually detect, and how many instances that we detect are actually Positive. In some scenarios we want to optimize precision, in some we want to optimize recall, but this is only when we __can't do both__ (because of lack of data, unbalanced instances, poor choice of classifiers, etc). In general, we actually want both high recall and high precision. To sort of squeeze this into one metric, we use the 'harmonic mean' (approximately average) between precision and recall, called the $F$ score or $F_1$ score:

$F_1 = 2 \cdot \frac{\text{precision} \cdot \text{recall}}{\text{precision} + \text{recall}}$

#### Tasks

- Make a confusion matrix.
- Calculate accuracy, precision, recall and f1 score.
- How well does this classifier do on your prediction task (compared to a baseline)?
- Think of a task where optimizing precision is important, and one where recall is important.


## Take-Home Assignment: MNIST: Know your Trade-offs

In the last three practicals we have focussed on understanding the basics of data mining: interpreting your data, creating intuitions, evaluating models, and criticizing the output. This week, you are tasked to an actual data challenge. A very old one.

### 0.1 - Preparing your Data

The file will be available on blackboard, under `mnist.csv`. Carefully read the description here or inform yourself a bit further by looking the dataset up on the internet.

### 0.2 - Set-up

Although not obligatory, or really necessary, having a team might benefit your result. More people means more laptops, means more people to run configurations and trying to improve scores. One of the things you might want to do is making a shared document including a table with ran experiments and their configurations, and what the final score was.

### 1 - Getting Started

The MNIST dataset consists of pictures of handwritten digits (see below).

![img](http://theanets.readthedocs.io/en/latest/_images/mnist-digits-small.png)

The feature vectors represent the (grayscale) intensity of every pixel in these 28x28 images (therefore 784 pixels). The class (label) represents the actual number, from 0 to 9. There are 60.000 of these labelled images available, which means it is a very big dataset (105 MB).

### 2 - Your Task

Your task is to minimize the error of your classifier (100 - accuracy), so that it would still perform well on new hand-written digits. Note: up until now we have looked at accuracy (correctly classified instances), now we will look at *incorrectly* classified instances (i.e. error). The state-of-the-art performance error is as low as 0.21 percent (that would look like 0.0021 in Orange). You are not expected to match this anywhere near (as the techniques to achieve this are simply not implemented in Orange). Use your knowledge of evaluation and parameter tuning you have acquired up until now to build a model you think will perform well.

### 3 - Tradeoffs

The dataset is very big. Normal machines will take a *lot* of time to get through training and testing of all the instances. To counter this, you can decide to sample part of the dataset. In Orange this can be done with the `Sample Data` widget.

**Important note:** removing data speeds up the classification process; however, it also means you will see less instances. As a result, it will likely hurt the generalization of your classifier. For this assignment, you will have to balance (i.e. trade-off) computation time (mostly your time) and performance.

### 4 - Reporting

It's good to keep a log with the steps you have taken: the amount of data you have sampled, which hyper-parameters you tuned to which values, and how this affected your results. If you feel comfortable with your solution, you can submit your set-up (orange workflow file) on blackboard. You have until the next practical to hand these in. Please include in the description **ONLY**:

- Sample: ... % (Percentage of data you have sampled).
- Proc: remove missing values, ...., (Preprocessing steps).
- Param: k=..., ... (The parameter settings of that classifier that yielded the lowest error).
- Result: 0.something (Your error score).

If you worked in a group, include the names of your group members.

Good luck!